<a href="https://colab.research.google.com/github/anassohail99/langchain/blob/main/AI_202_Project2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install -qU langchain pinecone-client google-generativeai openai tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.3/472.3 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 5.3 MB/s eta 0:00:00


In [3]:
from google.colab import userdata

import getpass
import os


if "PINECONE_API_KEY" not in os.environ:
    os.environ["PINECONE_API_KEY"] = userdata.get('PINECONE_API_KEY')

if "PINECONE_ENV" not in os.environ:
    os.environ["PINECONE_ENV"] = userdata.get('PINECONE_ENV')

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [44]:
from pinecone import Pinecone, ServerlessSpec
import time

api_key = os.environ.get("PINECONE_API_KEY")
# initialize the pinecone db
pc = Pinecone(api_key=api_key)
index_name = "gemini-rag-index-2"

# create pinecone index if not exists else connects to it
existing_indexes = [index_info['name'] for index_info in pc.list_indexes()]
print("existing_indexes: ",existing_indexes)

if index_name not in existing_indexes:
    pc.create_index(name=index_name,dimension=768,metric="cosine",spec=ServerlessSpec(
            cloud='aws',
            region='us-east-1'
        ))
    while not pc.describe_index(index_name).status["ready"]:
          time.sleep(1)

# connecting to the found index
index = pc.Index(index_name)
print(index.describe_index_stats())


existing_indexes:  ['gemini-rag-index', 'python-rag-project']
{'dimension': 768,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}


In [26]:
pip install q langchain_google_genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 2.7 MB/s eta 0:00:00


In [45]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings


# Embedding models like text-embedding-004 are commonly used in:

# Search & Retrieval – Find similar documents based on meaning.
# Chatbots & LLMs – Enhance memory and context retrieval.
# Recommendation Systems – Suggest similar items based on text similarity.


embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")
vector = embeddings.embed_query("hello, world!")
vector[:5]

[0.014134909026324749,
 -0.022324152290821075,
 -0.054603420197963715,
 -0.006284549366682768,
 -0.03392402455210686]

In [29]:
pip install qU langchain_community

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 42.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.9/78.9 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.1 MB/s eta 0:00:00
  Created wheel for qU: filename=qu-1.2.5-py3-none-any.whl size=3821 sha256=25af8b2583ad80001a3bc67620eb21df434f86cc205d1c4150a0ea3d8565a606
  Stored in directory: /root/.cache/pip/wheels/70/e5/a6/5bca7c06108e1fa9b94598d2a9d28b84cad178a8a7dcdee2c5
Successfully built qU


In [46]:
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter



# Handling Large Text – Many LLMs (like GPT) have a token limit. Splitting prevents truncation.
# Improved Retrieval – Helps vector databases (e.g., Pinecone, FAISS) store and retrieve information efficiently.
# Better Context for AI – Ensures AI models get relevant context by overlapping text chunks.


# Load documents
loader = TextLoader("testData.txt")  # Replace with your file
documents = loader.load()
# print("Document Content: ",documents[0].page_content)

# Split documents into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = text_splitter.split_documents(documents)

# print("Chunked Document Content: ",docs[0].page_content)


In [47]:
from tqdm import tqdm

# Create embeddings and upload to Pinecone
for doc in tqdm(docs):
    # Calls the embed_query() method on the document's text (doc.page_content).
    print(doc)
    vector = embeddings.embed_query(doc.page_content)
    # inserts or updates data in the Pinecone vector database.
    index.upsert([
        (doc.metadata["source"], vector, {"text": doc.page_content})
    ])

  0%|          | 0/10 [00:00<?, ?it/s]

page_content='Artificial Intelligence (AI) has revolutionized the way we interact with technology. It encompasses a wide range of applications, from simple rule-based systems to advanced machine learning models that can make autonomous decisions. The rise of AI can be attributed to increased computational power, the availability of big data, and advances in deep learning techniques.' metadata={'source': 'testData.txt'}


 10%|█         | 1/10 [00:00<00:03,  2.41it/s]

page_content='One of the most prominent areas of AI is natural language processing (NLP), which enables computers to understand, interpret, and generate human language. Virtual assistants like Siri, Alexa, and Google Assistant rely on NLP to respond to user queries. Additionally, AI-driven chatbots are widely used in customer support to enhance user experience.' metadata={'source': 'testData.txt'}


 20%|██        | 2/10 [00:00<00:03,  2.61it/s]

...

 80%|████████  | 8/10 [00:02<00:00,  3.47it/s]

page_content='As AI continues to evolve, researchers are exploring ways to make models more transparent and interpretable. Explainable AI (XAI) aims to provide insights into how AI systems make decisions, fostering trust and accountability.' metadata={'source': 'testData.txt'}


 90%|█████████ | 9/10 [00:02<00:00,  3.55it/s]

page_content='In conclusion, artificial intelligence is transforming multiple industries, enhancing efficiency, and improving human lives. While challenges exist, continued research and responsible AI development will pave the way for a future where AI-driven technologies benefit society as a whole.' metadata={'source': 'testData.txt'}


100%|██████████| 10/10 [00:03<00:00,  3.22it/s]


In [51]:
pip install qU langchain_pinecone

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 427.3/427.3 kB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.3/50.3 kB 4.0 MB/s eta 0:00:00
  Attempting uninstall: pinecone-plugin-inference
    Found existing installation: pinecone-plugin-inference 1.1.0
    Uninstalling pinecone-plugin-inference-1.1.0:
      Successfully uninstalled pinecone-plugin-inference-1.1.0
  Attempting uninstall: aiohttp
    Found existing installation: aiohttp 3.11.12
    Uninstalling aiohttp-3.11.12:
      Successfully uninstalled aiohttp-3.11.12
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pinecone-client 5.0.1 requires pinecone-plugin-inference<2.0.0,>=1.0.3, but you have pinecone-plugin-inference 3.1.0 whic

In [54]:
from langchain_pinecone import PineconeVectorStore

# embedding: An embedding function or model that converts text data into vector representations.
# This is essential for both adding new data to the vector store and querying
# it.

vectorstore = PineconeVectorStore(index_name=index_name, embedding=embeddings, text_key="text")

In [58]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    temperature=0.7,
)

In [59]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",  # Other options: "map_reduce", "refine"
    retriever=vectorstore
)

ValidationError: 1 validation error for RetrievalQA
retriever
  Input should be a valid dictionary or instance of BaseRetriever [type=model_type, input_value=<langchain_pinecone.vecto...bject at 0x7b102f5a7c50>, input_type=PineconeVectorStore]
    For further information visit https://errors.pydantic.dev/2.10/v/model_type